##### **Generating Image from text**

**Importing importing libraries**

In [2]:
import boto3
import json
import base64
import io
import uuid
import random
from PIL import Image

**Setting up Amazon Bedrock client**

In [3]:
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"
)

**Defining Model id and Prompt**

In [4]:
# Define the model ID
model_id = "amazon.nova-canvas-v1:0"

# Get user input for the image generation prompt.
prompt = input("Enter your image description: ")

**Creating Request Payload and Invoking the model**

In [5]:
# Generate a random seed between 0 and 858,993,459
seed = random.randint(0, 858993460)


body = json.dumps({
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "height": 1024,
            "width": 1024,
            "cfgScale": 8.0,
            "seed": seed
        }
    })

accept = "application/json"
content_type = "application/json"

response = bedrock_client.invoke_model(
    modelId=model_id,
    body=body,
    accept=accept, 
    contentType=content_type
)

**Processing the response and image returned in the response**

In [6]:
result = json.loads(response["body"].read())

print(result)

{'images': ['...']}


In [7]:
base64_image = result.get("images")[0]
base64_bytes = base64_image.encode('ascii')
image_bytes = base64.b64decode(base64_bytes)

# io.BytesIO(image_bytes) → Creates an in-memory buffer to store the image.
# Image.open() → Loads the image from memory without saving it to disk.

image = Image.open(io.BytesIO(image_bytes))
image.show()
unique_id = uuid.uuid4()


image.save(f"C:/Users/VrajDesai/Desktop/ML_AI_Course/AWS Practice Blogs/AWS-Course-Blog-2---Generating-and-Evaluating-Images-in-Bedrock/generated-images/{unique_id}.png")

print(f"Generated Image is Stored Successfully at : C:/Users/VrajDesai/Desktop/ML_AI_Course/AWS Practice Blogs/AWS-Course-Blog-2---Generating-and-Evaluating-Images-in-Bedrock/generated-images/{unique_id}.png")


Generated Image is Stored Successfully at : C:/Users/VrajDesai/Desktop/ML_AI_Course/AWS Practice Blogs/AWS-Course-Blog-2---Generating-and-Evaluating-Images-in-Bedrock/generated-images/62265cc6-0fe0-4198-9a57-9e8c09af448d.png


##### **Putting the Generated Image into S3 bucket and returning the S3 URL to check the image out**

**Creating a S3 bucket client**

In [8]:
s3_client = boto3.client("s3")

**Writing the bucket name**

In [11]:
bucket_name = "generated-images-blog2-bucket"

image_filename = f"generated-images/{unique_id}.png"

s3_client.put_object(
    Bucket = bucket_name,
    Key = image_filename,
    Body = image_bytes,
    ContentType = "image/png" # This ensures that image displays in the browser
)

s3_url = f"https://{bucket_name}.s3.amazonaws.com/{image_filename}"

print(f"The Generated image is available at: {s3_url}")

The Generated image is available at: https://generated-images-blog2-bucket.s3.amazonaws.com/generated-images/62265cc6-0fe0-4198-9a57-9e8c09af448d.png


##### **Passing the generated image again into the model and new prompt and getting the modified image**
 

**Getting the new prompt from the user, getting the existing image stored in the bucket and passing these values into the body payload**

In [29]:
# New prompt for modifying the image
new_prompt = input("Enter How you want to update the image: ")

# Read the existing image from local storage
existing_image_path = f"C:/Users/VrajDesai/Desktop/ML_AI_Course/AWS Practice Blogs/AWS-Course-Blog-2---Generating-and-Evaluating-Images-in-Bedrock/generated-images/{unique_id}.png"

with open(existing_image_path, "rb") as img_file:
    base64_image = base64.b64encode(img_file.read()).decode("utf-8")

# Prepare the request payload for Titan Image Generator G1 v2
body = json.dumps({
    "taskType": "IMAGE_VARIATION",  # Using image-to-image variation
    "imageVariationParams": {
        "text" : new_prompt,
        "images": [base64_image],  # Providing the input image as base64
        "similarityStrength" : 0.9
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "height": 1024,
        "width": 1024,
        "cfgScale": 8.0,
        "seed" : seed,
    }
})

**Invoking the model and getting the response and decoding the image**

In [30]:
# Invoke the Titan model
response = bedrock_client.invoke_model(
    modelId="amazon.nova-canvas-v1:0",
    body=body,
    accept="application/json",
    contentType="application/json"
)

# Process the modified image response
result = json.loads(response["body"].read())
base64_modified_image = result.get("images")[0]
image_bytes = base64.b64decode(base64_modified_image.encode('ascii'))

# Convert to Image
image = Image.open(io.BytesIO(image_bytes))
image.show()

**Saving the modified image locally into the updated-images folder**

In [31]:
# Save the modified image locally
updated_unique_id = uuid.uuid4()
updated_image_path = f"C:/Users/VrajDesai/Desktop/ML_AI_Course/AWS Practice Blogs/AWS-Course-Blog-2---Generating-and-Evaluating-Images-in-Bedrock/updated-images/{updated_unique_id}.png"
image.save(updated_image_path)

print(f"Updated Image Saved Successfully at: {updated_image_path}")


Updated Image Saved Successfully at: C:/Users/VrajDesai/Desktop/ML_AI_Course/AWS Practice Blogs/AWS-Course-Blog-2---Generating-and-Evaluating-Images-in-Bedrock/updated-images/dcd528d1-28e4-4473-ad0c-953c14edb576.png


**Uploading the image to S3 bucket**

In [ ]:

# Upload the updated image back to S3
updated_s3_key = f"updated-images/{updated_unique_id}.png"
s3_client = boto3.client("s3")

s3_client.put_object(
    Bucket=bucket_name,
    Key=updated_s3_key,
    Body=image_bytes,
    ContentType="image/png"
)

updated_s3_url = f"https://{bucket_name}.s3.amazonaws.com/{updated_s3_key}"
print(f"The Updated image is available at: {updated_s3_url}")


The Updated image is available at: https://generated-images-blog2-bucket.s3.amazonaws.com/updated-images/dcd528d1-28e4-4473-ad0c-953c14edb576.png
